In [ ]:
import xarray as xr
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gliderad2cp import process_currents, process_shear, process_bias, tools, download_example_data
from pathlib import Path
import cmocean.cm as cmo
data_dir = Path("data/raw_voto")
if not data_dir.exists():
    data_dir.mkdir(parents=True)

### Select a mission

You can explore out glider missions on the observations portal https://observations.voiceoftheocean.org/missions

Make sure it's one that has ADCP data! See a list of available ADCP datasets here https://erddap.observations.voiceoftheocean.org/erddap/files/ad2cp/

Or uncomment the last line in the cell below

In [ ]:
adcp_missions = pd.read_csv("https://erddap.observations.voiceoftheocean.org/erddap/tabledap/ad2cp.csvp")
adcp_missions["deployment_id"] = adcp_missions["name"].str.split('.', expand=True)[0]
# print(adcp_missions["deployment_id"].values) # uncomment to print available missions

In [ ]:
glider = "SEA044"
mission = 106
deployment_id = f"{glider}_M{mission}"

In [ ]:
if not deployment_id in adcp_missions["deployment_id"].values:
    print(f"ERROR: user speficified {deployment_id} not a valid deployement with ADCP data")

In [ ]:
url_glider = f"https://erddap.observations.voiceoftheocean.org/erddap/files/delayed_{deployment_id}/mission_timeseries.nc"
url_ad2cp = f"https://erddap.observations.voiceoftheocean.org/erddap/files/ad2cp/{deployment_id}.ad2cp.00000.nc"
data_file = data_dir / f"{deployment_id}.nc"
adcp_file = data_dir / f"{deployment_id}.ad2cp.00000.nc"

In [ ]:
if not data_file.exists():
    response_glider = requests.get(url_glider)
    with open(data_file, "wb") as f:
        f.write(response_glider.content)
if not adcp_file.exists():
    response_ad2cp = requests.get(url_ad2cp)
    with open(adcp_file, "wb") as f:
        f.write(response_ad2cp.content)

### Adjust processing options for gliderad2cp

In [ ]:
options = tools.get_options(xaxis=3, yaxis=None, shear_bias_regression_depth_slice=(10,1000))

### Process shear

In [ ]:
ds_adcp = process_shear.process(str(adcp_file), data_file, options)

### Get pre and post-dive GPS for DAC calculation

In [ ]:
data = xr.open_dataset(data_file)
dead = data.dead_reckoning
# Keep only data points with valid time, lon and lat
lon_lat_time = ~np.isnan(data.time) * ~np.isnan(data.longitude) * ~np.isnan(data.latitude)
dead = dead[lon_lat_time]
sample_step = ((dead.time.max() - dead.time.min()) / len(dead)) / np.timedelta64(1, 's')
target_step = 10  # target one sample every 10 seconds for the calculation of DAC. Coarsening speeds this up *a lot*
subsample = max(1, int(target_step / sample_step))
print(f"subsampling dead reckoning data by a factor of {subsample} to match target sampling rate of {target_step} seconds during DAC caclulation")
dead = dead[::subsample]

# dead reckoning 0 when lon/lat are from GPS fix. 1 when interpolated. Use the gradient of this to find
# where the glider starts & ends surface GPS fixes
dead_reckoning_post_change = dead[1:][dead.diff(dim='time') != 0]
post_dive = dead_reckoning_post_change[dead_reckoning_post_change == 0]

dead_reckoning_pre_change = dead[:-1][dead.diff(dim='time', label='lower') != 0]
pre_dive = dead_reckoning_pre_change[dead_reckoning_pre_change == 0]

# Cut out any predives after the last postdives and postdives before the first predive
pre_dive = pre_dive[pre_dive.time < post_dive.time.max()]
post_dive = post_dive[post_dive.time > pre_dive.time.min()]

gps_predive = np.array([[time, lat, lon] for time, lat, lon in
               zip(pre_dive.time.values, pre_dive.latitude.values, pre_dive.longitude.values)])
gps_postdive = np.array([[time, lat, lon] for time, lat, lon in
                zip(post_dive.time.values, post_dive.latitude.values, post_dive.longitude.values)])
dive_time_hours = (post_dive.time.values - pre_dive.time.values) / np.timedelta64(1, 'h')
assert (dive_time_hours > 0).all
assert 24 > np.mean(dive_time_hours) > 0.5



### Calculate DAC

In [ ]:
currents, DAC = process_currents.process(
    ds_adcp, gps_predive, gps_postdive, options
)

### Estimate and calculate shear bias

This optional processing step attempts to correct for along-beam shear bias. Shear-bias is described in Todd et al. 2017 (JAOTECH, https://doi.org/10.1175/JTECH-D-16-0156.1), section 
    "Shear-bias is the result of very small shear present in beams during individual pings. It is not yet known what causes it although reports from Nortek and others suggest an instrument dependence. It is also reduced with stricter signal-to-noise ratio thresholds. As shear is exagerated throughout the water-column, the error in velocity grows with the depth of the profile. It is generally visible as an erroneous supplement velocity component aligned with the glider's direction of travel


In [ ]:
currents = process_bias.process(currents, options)

### Compare output

The following three plots contrast the eastward velocities estimates at three crutical points of processing:

1. From integration of shear
2. After referencing integrated shear profiles to DAC
3. After applying the shear bias correction to DAC referenced velocities


In [ ]:
variables = ["velocity_E_no_reference", "velocity_E_DAC_reference", "velocity_E_DAC_reference_sb_corrected"]
titles = ["No referencing", "DAC referencing", "DAC referencing, bias correction"]
fig, axs = plt.subplots(3, 1, figsize=(10,14))
for i in range(3):
    ax = axs[i]
    mappable = ax.pcolormesh(currents.time[:-1], currents.depth, currents[variables[i]][:, :-1], cmap=cmo.balance, vmin=-0.6, vmax=0.6)
    ax.invert_yaxis()
    fig.colorbar(ax=ax,mappable=mappable, label='Eastward velocity (m/s)')
    ax.set(ylabel='Depth (m)', title=titles[i])

### Write out data

In [ ]:
currents.to_netcdf(f"data/{deployment_id}_gridded_with_adcp.nc")